In [1]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import os
import torch
from PIL import Image

In [2]:
class ImageFolderWithLabels(Dataset):
    def __init__(self, root_dir, transform=None, half_size=False):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self._get_image_paths_and_labels(half_size)

    def _get_image_paths_and_labels(self, half_size=False):
      for subfolder in ["real", "fake"]:
            subfolder_path = os.path.join(self.root_dir, subfolder)
            image_list = os.listdir(subfolder_path)  # Get all filenames

            if half_size:
                image_list = image_list[::2]  # Select every alternate image

            for filename in image_list:
                if filename.endswith(".jpg"):
                    img_path = os.path.join(subfolder_path, filename)
                    self.image_paths.append(img_path)
                    self.labels.append(0 if subfolder == "real" else 1)  # Assign label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, label

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet statistics
])

In [4]:
# Load data from each directory
train_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/celebDF_processed_split', 'Train'), transform=transform, half_size=True)
test_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/celebDF_processed_split', 'Test'), transform=transform, half_size=True)
val_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/celebDF_processed_split', 'val'), transform=transform, half_size=True)

In [5]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

4723
590
592


In [6]:
BATCH_SIZE=32

In [7]:
# Create dataloaders for training, testing, and validation
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)  # Adjust batch size as needed
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
maxvit = torchvision.models.maxvit_t(weights='DEFAULT')

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
total_params = sum(p.numel() for p in maxvit.parameters())
total_params

30919624

In [10]:
import torch.optim as optim
optimizer = optim.Adam(maxvit.parameters(), lr=0.001)

In [11]:
# Define your loss function
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [12]:
# Define the device to run the model on (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
maxvit.to(device)

MaxVit(
  (stem): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
    )
    (1): Conv2dNormActivation(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (blocks): ModuleList(
    (0): MaxVitBlock(
      (layers): ModuleList(
        (0): MaxVitLayer(
          (layers): Sequential(
            (MBconv): MBConv(
              (proj): Sequential(
                (0): AvgPool2d(kernel_size=3, stride=2, padding=1)
                (1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
              )
              (stochastic_depth): Identity()
              (layers): Sequential(
                (pre_norm): BatchNorm2d(64, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
                (conv_a): Conv2dNormActivation(
           

In [14]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=8):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        # Training phase
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Calculate training loss and accuracy
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct / total

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                val_running_loss += val_loss.item() * val_inputs.size(0)
                _, val_predicted = val_outputs.max(1)
                val_total += val_labels.size(0)
                val_correct += val_predicted.eq(val_labels).sum().item()

        # Calculate validation loss and accuracy
        epoch_val_loss = val_running_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct / val_total

        # Print epoch statistics
        print(f"Epoch {epoch + 1}/{num_epochs}",
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}",
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    print('Training complete')


num_epochs = 10


train_model(maxvit, train_loader, val_loader, criterion, optimizer, device, num_epochs)


Epoch 1/10 Train Loss: 0.8080, Train Acc: 0.5475 Val Loss: 0.6316, Val Acc: 0.6368
Epoch 2/10 Train Loss: 0.6924, Train Acc: 0.5717 Val Loss: 0.6292, Val Acc: 0.6334
Epoch 3/10 Train Loss: 0.6014, Train Acc: 0.6703 Val Loss: 0.5472, Val Acc: 0.7348
Epoch 4/10 Train Loss: 0.4803, Train Acc: 0.7665 Val Loss: 0.4923, Val Acc: 0.7669
Epoch 5/10 Train Loss: 0.3573, Train Acc: 0.8404 Val Loss: 0.2884, Val Acc: 0.8750
Epoch 6/10 Train Loss: 0.2741, Train Acc: 0.8795 Val Loss: 0.2537, Val Acc: 0.9071
Epoch 7/10 Train Loss: 0.1928, Train Acc: 0.9236 Val Loss: 0.1927, Val Acc: 0.9291
Epoch 8/10 Train Loss: 0.1758, Train Acc: 0.9272 Val Loss: 0.1751, Val Acc: 0.9257
Epoch 9/10 Train Loss: 0.1419, Train Acc: 0.9490 Val Loss: 0.2074, Val Acc: 0.9223
Epoch 10/10 Train Loss: 0.1013, Train Acc: 0.9602 Val Loss: 0.2407, Val Acc: 0.9071
Training complete


In [15]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # No need to calculate gradients during testing
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / len(test_loader.dataset)
    test_acc = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [16]:
test_model(maxvit, test_loader, criterion, device)

Test Loss: 0.1926, Test Accuracy: 0.9203


In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion_mat = confusion_matrix(true_labels, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion_mat)

#Assuming you have already defined your test_loader and model
evaluate_model(maxvit, test_loader, device)



Accuracy: 0.9203
Precision: 0.9804
Recall: 0.8562
F1 Score: 0.9141
Confusion Matrix:
[[293   5]
 [ 42 250]]
